In [128]:
import sklearn
import numpy as np
import pandas as pd
import optuna
from datetime import datetime
from IPython.utils import io

from sklearn.impute import SimpleImputer
from sklearn.neighbors import LocalOutlierFactor, KNeighborsRegressor
from sklearn.feature_selection import (
    RFE,
    RFECV, 
    VarianceThreshold, 
    SelectKBest, 
    mutual_info_regression, 
    f_regression, 
    SelectFromModel,
    f_classif,
    chi2
)

from sklearn.linear_model import Ridge, Lasso, LinearRegression, BayesianRidge
from sklearn.preprocessing import QuantileTransformer, RobustScaler, MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score
from sklearn.model_selection import RepeatedKFold, RepeatedStratifiedKFold, train_test_split, cross_val_score
from sklearn.svm import SVR, LinearSVR
from sklearn.neural_network import MLPRegressor

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RationalQuadratic, WhiteKernel, Matern, ExpSineSquared, DotProduct, RBF

from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.ensemble import (
    ExtraTreesRegressor, 
    RandomForestRegressor, 
    BaggingRegressor, 
    AdaBoostRegressor,
    StackingRegressor
)

from xgboost import XGBRegressor

## TaskSolver object

In [76]:
class TaskSolver:
    def __init__(
        self, 
        train_df,
        test_df,
        imputer,
        outliers_detector,
        scaler,
        corr_threshold,
    ):
        self.train_df = train_df
        self.train_features = train_df.drop(["id", "y", "class", "fold"], axis=1).to_numpy()
        self.train_targets = train_df["y"]
        self.train_folds = train_df["fold"]
        self.test_df = test_df
        self.test_features = test_df.drop(["id"], axis=1).to_numpy()
        
        self.imputer = imputer
        self.outliers_detector = outliers_detector
        self.scaler = scaler
        self.corr_threshold = corr_threshold
    
    def _count_missing_values(self, x):
        missing_count = np.count_nonzero(np.isnan(x))
        total_entries = x.shape[0] * x.shape[1]
        missing_pct = missing_count / total_entries
        print(f"Total missing values: {missing_count}/{total_entries} ({missing_pct*100:.3f}%)")
        return missing_pct

    def _handling_missing_values(self):
        self._count_missing_values(self.train_features)
        self._count_missing_values(self.test_features)
        print("Imputing missing values...")
        
        self.imputer.fit(self.train_features)
        self.train_features = self.imputer.transform(self.train_features)
        self.test_features = self.imputer.transform(self.test_features)
        assert self._count_missing_values(self.train_features) == 0.0
        assert self._count_missing_values(self.test_features) == 0.0
    
    def _detect_and_remove_outliers(self):
        outliers = self.outliers_detector.fit_predict(self.train_features)
        outliers_count = sum(outliers == -1)
        outliers_pct = outliers_count / self.train_features.shape[0]
        print(f"{outliers_count} outliers detected! ({outliers_pct*100:.3f}%)")

        print("Removing outliers...")
        self.train_features = self.train_features[outliers == 1]
        self.train_targets = self.train_targets[outliers == 1]
        self.train_folds = self.train_folds[outliers == 1]
        print(
            "Train shape:", self.train_features.shape, 
            "Train target shape:", self.train_targets.shape,
            "Train fold shape:", self.train_folds.shape,
        )
    
    def _normalize_data(self):
        print("Rescaling data...")
        self.scaler.fit(self.train_features)
        self.train_features = self.scaler.transform(self.train_features)
        self.test_features = self.scaler.transform(self.test_features)
    
    def _corr2_coeff(self, A, B):
        # credit: https://stackoverflow.com/a/30143754/7053239
        # Rowwise mean of input arrays & subtract from input arrays themeselves
        A_mA = A - A.mean(1)[:, None]
        B_mB = B - B.mean(1)[:, None]
        # Sum of squares across rows
        ssA = (A_mA ** 2).sum(1)
        ssB = (B_mB ** 2).sum(1)
        # Finally get corr coeff
        return np.dot(A_mA, B_mB.T) / np.sqrt(np.dot(ssA[:, None], ssB[None]))

    def _remove_redundant_features(self):
        print("Removing zero-variance features...")
        self.var_selector = VarianceThreshold(threshold=0)
        self.var_selector.fit(self.train_features)
        self.train_features = self.var_selector.transform(self.train_features)
        self.test_features = self.var_selector.transform(self.test_features)
        print("Train shape:", self.train_features.shape, "Test shape:", self.test_features.shape)
        
        print("Removing highly correlated features...")
        cor = self._corr2_coeff(self.train_features.T, self.train_features.T)
        correlated_pairs = np.argwhere(np.triu(np.absolute(cor) >= self.corr_threshold, 1))
        self.train_features = np.delete(self.train_features, correlated_pairs[:, 1], axis=1)
        self.test_features = np.delete(self.test_features, correlated_pairs[:, 1], axis=1)
        print("Train shape:", self.train_features.shape, "Test shape:", self.test_features.shape)

    def preprocessing(self):
        self._handling_missing_values()
        self._detect_and_remove_outliers()
        self._normalize_data()
        self._remove_redundant_features()
    
    def feature_selection(self, selector):
        print("Selecting features...")
        self.feature_selector = selector
        self.feature_selector.fit(self.train_features, self.train_targets)
        self.train_features = self.feature_selector.transform(self.train_features)
        self.test_features = self.feature_selector.transform(self.test_features)
        print("Train shape:", self.train_features.shape, "Test shape:", self.test_features.shape)

    def _round_to_nearest_int(self, x):
        decimal = x % 1
        mask = abs(x - np.round(x)) <= self.rounding_threshold
        x[mask] = np.round(x[mask])
        return x

    def _cross_validation(self, model, scoring):
        scores = []
        self.oof_preds = []
        self.oof_targets = []
        for fold_nr in range(5):
            val_mask = (self.train_folds == fold_nr)
            x_train = self.train_features[~val_mask]
            y_train = self.train_targets[~val_mask]
            x_val = self.train_features[val_mask]
            y_val = self.train_targets[val_mask]
            model = model.fit(x_train, y_train)
            
            pred_val = model.predict(x_val)
            if self.rounding_threshold is not None:
                pred_val = self._round_to_nearest_int(pred_val)

            r2 = r2_score(y_val, pred_val)
            scores.append(r2)
            self.oof_preds.extend(pred_val.tolist())
            self.oof_targets.extend(y_val.tolist())

        return scores
    
    def _save_oof_prediction(self):
        oof = pd.DataFrame()
        oof["y_pred"] = self.oof_preds
        oof["y"] = self.oof_targets
        oof.to_csv(f"submissions/{self.timestamp}.oof.csv", index=False)

    def train(self, model, scoring="r2", rounding_threshold=None, verbose=False):
        self.timestamp = datetime.now().strftime("%Y_%m_%d-%I:%M:%S")
        self.rounding_threshold = rounding_threshold
        scores = self._cross_validation(model, scoring)

        if verbose:
            print("R2: {:.5f} ± {:.5f}".format(np.mean(scores), np.std(scores)))
        
        return np.mean(scores)

    def predict_test(self, model):
        model.fit(self.train_features, self.train_targets)
        pred = model.predict(self.test_features)
        if self.rounding_threshold is not None:
            pred = self._round_to_nearest_int(pred)
        submission = pd.DataFrame()
        submission["id"] = self.test_df["id"]
        submission["y"] = pred
        submission.to_csv(f"submissions/{self.timestamp}.csv", index=False)
        self._save_oof_prediction()

## Run

### Use Optuna to tune the data preprocessing pipeline

In [120]:
SEED = 42

# selector = RandomForestRegressor(
#     **{
#         'bootstrap': False,
#         'max_depth': 50,
#         'max_features': 'log2',
#         'min_samples_leaf': 1,
#         'min_samples_split': 8,
#         'n_estimators': 553,
#     },
#     random_state=SEED
# )
selector = ExtraTreesRegressor(random_state=SEED)

solver = TaskSolver(
    train_df=pd.read_csv("data/kfold.csv"),
    test_df=pd.read_csv("data/X_test.csv"),
    imputer=IterativeImputer(
        max_iter=10,
        n_nearest_features=50, 
        random_state=SEED, 
        verbose=0
    ),
    outliers_detector=LocalOutlierFactor(
        n_neighbors=83, 
        algorithm="auto", 
        contamination="auto"
    ),
    scaler=QuantileTransformer(output_distribution="normal"),
    corr_threshold=0.8424737082383797,
)

solver.preprocessing()

Total missing values: 76902/1008384 (7.626%)
Total missing values: 40634/645632 (6.294%)
Imputing missing values...
Total missing values: 0/1008384 (0.000%)
Total missing values: 0/645632 (0.000%)
101 outliers detected! (8.333%)
Removing outliers...
Train shape: (1111, 832) Train target shape: (1111,) Train fold shape: (1111,)
Rescaling data...
Removing zero-variance features...
Train shape: (1111, 828) Test shape: (776, 828)
Removing highly correlated features...
Train shape: (1111, 773) Test shape: (776, 773)


In [121]:
solver.feature_selection(SelectFromModel(RandomForestRegressor(
    **{
        'bootstrap': False,
        'max_depth': 50,
        'max_features': 'log2',
        'min_samples_leaf': 1,
        'min_samples_split': 8,
        'n_estimators': 553,
    },
    random_state=SEED
), threshold="mean"))

Selecting features...
Train shape: (1111, 144) Test shape: (776, 144)


In [122]:
solver.train(Ridge(alpha=20))

0.5551152222381928

In [131]:
def objective(trial):
    # model = XGBRegressor(
    #     **{
    #         'tree_method': 'auto',
    #         'lambda': trial.suggest_float(
    #             'lambda', 1e-3, 10.0
    #         ),
    #         'alpha': trial.suggest_float(
    #             'alpha', 1e-3, 10.0
    #         ),
    #         'colsample_bytree': trial.suggest_categorical(
    #             'colsample_bytree', [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    #         ),
    #         'subsample': trial.suggest_categorical(
    #             'subsample', [0.6, 0.7, 0.8, 1.0]
    #         ),
    #         'learning_rate': trial.suggest_categorical(
    #             'learning_rate', [0.008, 0.009, 0.01, 0.012, 0.014, 0.016, 0.018, 0.02]
    #         ),
    #         'n_estimators': trial.suggest_categorical(
    #             "n_estimators", [150, 200, 300, 3000]
    #         ),
    #         'max_depth': trial.suggest_categorical(
    #             'max_depth', [4, 5, 7, 9, 11, 13, 15, 17]
    #         ),
    #         'random_state': SEED,
    #         'min_child_weight': trial.suggest_int(
    #             'min_child_weight', 1, 300
    #         ),
    #     }
    # )

    # model = KNeighborsRegressor(
    #     **{
    #         "n_neighbors": trial.suggest_int("n_neighbors", 5, 100),
    #         "weights": trial.suggest_categorical("weights", ["uniform", "distance"])
    #     }
    # )

    # model = GaussianProcessRegressor(
    #     normalize_y=True,
    #     random_state=SEED,
    #     kernel=Matern(
    #         **{
    #             "length_scale": trial.suggest_float("length_scale", 1.0, 25.0),
    #             "nu": trial.suggest_float("nu", 0.5, 2.5),
    #         }
    #     )
    # )

    # model = GaussianProcessRegressor(
    #     normalize_y=True,
    #     random_state=SEED,
    #     kernel=RBF(
    #         **{
    #             "length_scale": trial.suggest_float("length_scale", 1.0, 100.0),
    #         }
    #     )
    # )

    # model = SVR(
    #     **{
    #         "C": trial.suggest_float("C", 1.0, 50.0),
    #         "kernel": "rbf",
    #         "gamma": trial.suggest_categorical("gamma", ["scale", "auto"]),
    #     }
    # )

    model = StackingRegressor(
        estimators=[
            ("ridge", Ridge(alpha=trial.suggest_float("ridge_alpha", 1e-3, 100))),
            ("svr", SVR(C=trial.suggest_float("C", 1.0, 50.0))),
            ("gp", GaussianProcessRegressor(
                normalize_y=True, 
                random_state=SEED, 
                kernel=Matern(length_scale=trial.suggest_float("length_scale", 1.0, 25.0), nu=1.5)))
        ],
        final_estimator=Ridge(alpha=trial.suggest_float("stack_alpha", 1e-3, 100))
    )

    cv_score = solver.train(model)
    return cv_score

study = optuna.create_study(
    direction="maximize",
    # sampler=optuna.samplers.TPESampler(seed=SEED),
    # pruner=optuna.pruners.MedianPruner(n_warmup_steps=10),
)

study.optimize(objective, n_trials=100)

[I 2022-11-14 02:00:08,022] A new study created in memory with name: no-name-cead4921-38b4-4fea-aafd-0818f509ee34
[I 2022-11-14 02:00:24,652] Trial 0 finished with value: 0.6387047053077517 and parameters: {'ridge_alpha': 53.834074959992094, 'C': 28.016094862195267, 'length_scale': 6.258753205381416, 'stack_alpha': 54.87090162750451}. Best is trial 0 with value: 0.6387047053077517.
[I 2022-11-14 02:00:43,650] Trial 1 finished with value: 0.63932111098796 and parameters: {'ridge_alpha': 22.60696263333599, 'C': 41.987803516921836, 'length_scale': 3.2388111666565926, 'stack_alpha': 0.32661539477321516}. Best is trial 1 with value: 0.63932111098796.
[I 2022-11-14 02:00:57,852] Trial 2 finished with value: 0.6393265462040341 and parameters: {'ridge_alpha': 93.33297297507436, 'C': 21.26279751209819, 'length_scale': 15.868355261287483, 'stack_alpha': 98.60027103950524}. Best is trial 2 with value: 0.6393265462040341.
[I 2022-11-14 02:01:11,625] Trial 3 finished with value: 0.6413945600429691 

In [132]:
study.best_params

{'ridge_alpha': 87.4058656055748,
 'C': 47.648587694460346,
 'length_scale': 23.18789935501063,
 'stack_alpha': 77.7221193768485}

In [133]:
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

In [134]:
plot_parallel_coordinate(study)

In [135]:
plot_param_importances(study)

### Train with the found optimal hyperparams

In [137]:
# model = GaussianProcessRegressor(
#     normalize_y=True,
#     random_state=SEED,
#     kernel=Matern(
#         **{
#             'length_scale': 11.367302557387221, 
#             'nu': 2.5
#         }
#     )
# )

# model = GaussianProcessRegressor(
#     normalize_y=True,
#     random_state=SEED,
#     kernel=RBF(
#         **{
#             'length_scale': 9.120813189282313,
#         }
#     )
# )

# model = GaussianProcessRegressor(
#     normalize_y=True,
#     random_state=SEED,
#     kernel=Matern(
#         **{
#             'length_scale': 7.10539712598952,
#             "nu": 1.5293470785583438,
#         }
#     )
# )

# model = GaussianProcessRegressor(
#     normalize_y=True,
#     random_state=SEED,
#     kernel=RBF(
#         **{
#             'length_scale': 1.8101745493955144,
#         }
#     )
# )
# model = SVR(
#     **{
#         "C": 49.99410203072422,
#         "gamma": "auto",
#     }
# )

model = StackingRegressor(
        estimators=[
            ("ridge", Ridge(alpha=87.4058656055748)),
            ("svr", SVR(C=47.648587694460346)),
            ("gp", GaussianProcessRegressor(
                normalize_y=True, 
                random_state=SEED, 
                kernel=Matern(length_scale=23.18789935501063, nu=1.5)))
        ],
        final_estimator=Ridge(alpha=77.7221193768485)
    )


# model = BayesianRidge()

In [138]:
solver.train(model)

/Users/kvu/opt/anaconda3/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning:

The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.

/Users/kvu/opt/anaconda3/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning:

The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.

/Users/kvu/opt/anaconda3/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning:

The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.

/Users/kvu/opt/anaconda3/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning:



0.6489497771525954

In [139]:
solver.predict_test(model)

/Users/kvu/opt/anaconda3/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning:

The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.

/Users/kvu/opt/anaconda3/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning:

The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.

/Users/kvu/opt/anaconda3/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning:

The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.

/Users/kvu/opt/anaconda3/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning:

